In [1]:
# Give some definitions (There is no need to change the following definitions)
import sys
import re
from typing import List, Dict, Union

import sympy as sym
from IPython.display import display
from IPython.display import Math

def is_num(s: str) -> bool:
    try:
        float(s)
    except ValueError:
        return False
    else:
        return True

def set_symbol_from_text(text: str) -> List[sym.Symbol]:
    """Make list of sympy symbols from a text
 
    Parameters
    ----------
    text : str
        Comma separated words

    Returns
    -------
    symbol_list : List[sym.Symbol]
        List of replaced symbols

    Examples
    -------
    input_variables = r'x_{1}, x_{2}'

    x = set_symbol_from_text(input_variables)
    """
    symbol_list = []
    for term in re.split(',\s*', text):
        symbol_list.append(sym.Symbol(term))
    return symbol_list

def replace_text_with_symbol(  
    text: str, c: List[sym.Symbol], x: List[sym.Symbol]
    ) -> str:
    """Make a replaced string with defined sympy symbols

    Parameters
    ----------
    text : str
        original string
    c : List[sym.Symbol]
        List of constants (sympy symbols)
    x: List[sym.symbol]
        List of variables (sympy symbols)

    Returns
    -------
    str
        replaced string
    """
    text = ' ' + text + ' '
    for i, v in enumerate(c):
        rep = r"\1c[{0}]\2".format(i)
        my_regex = r"([^a-zA-Z_0-9])" + re.escape(sym.latex(v)) + r"([^a-zA-Z_0-9])"
        while re.search(my_regex, text) != None:
            text = re.sub(my_regex, rep, text)
    for i, v in enumerate(x):
        rep = r"\1x[{0}]\2".format(i)
        my_regex = r"([^a-zA-Z_0-9])" + re.escape(sym.latex(v)) + r"([^a-zA-Z_0-9])"
        while re.search(my_regex, text) != None:
            text = re.sub(my_regex, rep, text)
    t = text.strip()
    text = t.replace(r"{", "").replace(r"}", "")
    return text

In [2]:
# Start scripts
sym.init_printing(use_unicode=True)

# Set output file name
output_filename = "van_der_Pol_event.inp"
# Set constants
input_const = r'\epsilon, \nu_{11}, \nu_{22}'
c = set_symbol_from_text(input_const)
# Set variables
input_var = r"x_{1}, x_{2}"
x = set_symbol_from_text(input_var)
# Set variables for initial positions
input_var_c = r"x_{1}^{\mathrm{c}}, x_{2}^{\mathrm{c}}"
xc = set_symbol_from_text(input_var_c)
# Display inputs for check
display('Defined constants')
display(Math(sym.latex(c)))
display('Defined variables')
display(Math(sym.latex(x)))
display('Defined constants for the initial positions')
display(Math(sym.latex(xc)))
# Add the list for 'xc' to the list 'c'
c = c + xc

'Defined constants'

<IPython.core.display.Math object>

'Defined variables'

<IPython.core.display.Math object>

'Defined constants for the initial positions'

<IPython.core.display.Math object>

In [3]:
# Set equations
# Variables, drift and diffusion coefficients must be enclosed in [].
Eqs = []
Eqs.append(r"d[x_{1}] = [x_{2}]*dt + [\nu_{11}]*d W_{1}")
Eqs.append(r"d[x_{2}] =" \
    + r"[\epsilon*(1-x_{1}**2)*x_{2} - x_{1}]*dt + [\nu_{22}]*d W_{2}")

# Extract strings for drift and diffusion
str_drift = []
str_diff = []
for eq in Eqs:
    result = re.findall(r'\[(.*?)\]', eq)
    if(len(result) != 3):
        print("The format of equation is not adequate: {0}".format(eq))
        sys.exit(1)
    str_drift.append(result[1])
    str_diff.append(result[2])

# Convert strings to sympy
drift = []
for ex in str_drift:
    drift.append(eval(replace_text_with_symbol(ex,c,x)))
drift = sym.Matrix(len(drift), 1, drift)
diff_vector = []
for ex in str_diff:
    diff_vector.append(eval(replace_text_with_symbol(ex,c,x)))
diff = []
for i, variable in enumerate(x):
    tmp_array = [0.0] * len(x)
    tmp_array[i] = diff_vector[i]
    diff.append(tmp_array)
diff = sym.Matrix(diff)

# Display input SDEs for check
latex_bold_x = sym.Symbol('\mathbf{x}')
latex_bold_W = sym.Symbol('\mathbf{W}')
display('Original stochastic differential equations')
for i in range(len(x)):
    latex_W = sym.Symbol('W_{0}'.format(i+1))
    print_sde = Math(r'd {0}(t) = \left({1}\right) dt + \left({2}\right) d {3}(t)'\
        .format(x[i], sym.latex(drift[i]), sym.latex(diff[i,i]), latex_W))
    display(print_sde)

'Original stochastic differential equations'

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [4]:
# Make sympy symbols for partial derivatives
deriv = []
for tmp_x in x:
    deriv.append(sym.Symbol('\\partial_{{{0}}}'.format(tmp_x)))

# Display the derived SDEs
print_sde = Math(r'd{0}(t) = {1} dt + {2} d {3}(t)'\
    .format(latex_bold_x, sym.latex(drift), 
    sym.latex(diff), latex_bold_W))
display('Extended stochastic differential equations')
display(print_sde)

'Extended stochastic differential equations'

<IPython.core.display.Math object>

In [5]:
# Make sympy symbols for partial derivatives
deriv = []
for tmp_x in x:
    deriv.append(sym.Symbol('\\partial_{{{0}}}'.format(tmp_x)))

# Derive the adjoint operator (backward Kolmogrov) and display it
adj_L = 0
B = diff * diff.transpose()

deriv = sym.Matrix([deriv])
latex_adj_L = sym.Symbol('\mathcal{L}^{\dagger}')
print_adj_L = ""
drift_terms = []
drift_derivs = []
for dri, der in zip(drift,deriv): # 1st order
    drift_terms.append(dri)
    drift_derivs.append(der)
    adj_L = adj_L + dri*der
    print_adj_L = print_adj_L \
        + '\\left({0}\\right) {1}'\
            .format(sym.latex(dri), sym.latex(der)) \
        + '+'
diff_terms = []
diff_derivs = []
for i in range(len(x)): # 2nd order
    for j in range(len(x)):
        if B[len(x)*i+j] != 0: 
            diff_terms.append(0.5*B[len(x)*i+j])
            diff_derivs.append(deriv[i]*deriv[j])
            adj_L = adj_L + 0.5*B[len(x)*i+j]*deriv[i]*deriv[j]
            print_adj_L = print_adj_L \
             + '\\frac{{1}}{{2}}\\left({0}\\right) {1}{2}'\
                  .format(sym.latex(B[len(x)*i+j]), \
                          sym.latex(deriv[i]), \
                           sym.latex(deriv[j])
                          ) \
             + '+'
print_adj_L = print_adj_L[:-1]  # Remove the final plus sign
print_dual = Math(r'{0} = {1}'.format(latex_adj_L, print_adj_L))
display('Derived adjoint operator')
display(print_dual)


'Derived adjoint operator'

<IPython.core.display.Math object>

In [6]:
# Apply variable transformations for coordinate shifts
for i, v in enumerate(x):
    for j, term in enumerate(drift_terms):
        drift_terms[j] = drift_terms[j].subs(v, v+xc[i])
    for j, term in enumerate(diff_terms):
        diff_terms[j] = diff_terms[j].subs(v, v+xc[i])

# Derive the adjoint operator (backward Kolmogrov) and display it
adj_L = 0
latex_adj_L = sym.Symbol('\mathcal{L}^{\dagger}')
print_adj_L = ""
for dri, der in zip(drift_terms,drift_derivs): # 1st order
    adj_L = adj_L + dri*der
    print_adj_L = print_adj_L \
        + '\\left({0}\\right) {1}'\
            .format(sym.latex(dri), sym.latex(der)) \
        + '+'
for diff, der in zip(diff_terms,diff_derivs): # 2nd order
    adj_L = adj_L + diff*der
    print_adj_L = print_adj_L \
        + '\\left({0}\\right) {1}'\
            .format(sym.latex(diff), sym.latex(der)) \
        + '+'

print_adj_L = print_adj_L[:-1]  # Remove the final plus sign
print_dual = Math(r'{0} = {1}'.format(latex_adj_L, print_adj_L))
display('Derived adjoint operator')
display(print_dual)

'Derived adjoint operator'

<IPython.core.display.Math object>

In [7]:
# Display constants (again) for setting values
display('Defined constants (again)')
display(Math(sym.latex(c)))

'Defined constants (again)'

<IPython.core.display.Math object>

In [8]:
# Print degrees for each term
adj_L = sym.expand(adj_L)
variables = []
variables.extend(x)
variables.extend(deriv)
variables.extend(c)
zero_list = [0]*len(x)

# Print terms (for check)
output = "# [State changes] / rate / [Indices for rate] / [Indices for constants] # term\n"
result = []
for t in adj_L.args:
    degree_list = list(sym.degree_list(t, gens=variables))
    change_list = []
    coeff_list = []
    const_list = []
    for i in range(len(x)):
        change_list.append(degree_list[i]-degree_list[i+len(x)])
        coeff_list.append(degree_list[i+len(x)])
    const_list = const_list + degree_list[2*len(x):]
    is_constant = all(elem == 0 for elem in degree_list)
    if is_constant == True:
        result.append([change_list, float(t), coeff_list, const_list, sym.latex(t)])
    else:
        result.append([change_list, float(sym.LC(t)), coeff_list, const_list, sym.latex(t)])
result = sorted(result)
for item in result:
    str0 = "{0}".format(item[0]).replace(' ', '')
    str1 = "{0}".format(item[1])
    str2 = "{0}".format(item[2]).replace(' ', '')
    str3 = "{0}".format(item[3]).replace(' ', '')
    str4 = "{0}".format(item[4])
    output = output + "{0} / {1} / {2} / {3} # {4}\n".format(str0, str1, str2, str3, str4)

state_change_tmp = [item[0] for item in result]
state_change = []
[x for x in state_change_tmp if x not in state_change and not state_change.append(x)]

# Add the basic information
output = "{0} {1} {2} {3}\n".format(len(adj_L.args), len(state_change), len(x), len(c)) + output
output = "# #terms #terms_without_duplicate #x #const / Constants: {0}\n".format(c) + output

with open(output_filename, mode='w') as f:
    f.write(output)

# For check
print(output)

# #terms #terms_without_duplicate #x #const / Constants: [\epsilon, \nu_{11}, \nu_{22}, x_{1}^{\mathrm{c}}, x_{2}^{\mathrm{c}}]
14 10 2 5
# [State changes] / rate / [Indices for rate] / [Indices for constants] # term
[-2,0] / 0.5 / [2,0] / [0,2,0,0,0] # 0.5 \nu_{11}^{2} \partial_{x_{1}}^{2}
[-1,0] / 1.0 / [1,0] / [0,0,0,0,1] # \partial_{x_{1}} x_{2}^{\mathrm{c}}
[-1,1] / 1.0 / [1,0] / [0,0,0,0,0] # \partial_{x_{1}} x_{2}
[0,-2] / 0.5 / [0,2] / [0,0,2,0,0] # 0.5 \nu_{22}^{2} \partial_{x_{2}}^{2}
[0,-1] / -1.0 / [0,1] / [0,0,0,1,0] # - \partial_{x_{2}} x_{1}^{\mathrm{c}}
[0,-1] / -1.0 / [0,1] / [1,0,0,2,1] # - \epsilon \partial_{x_{2}} \left(x_{1}^{\mathrm{c}}\right)^{2} x_{2}^{\mathrm{c}}
[0,-1] / 1.0 / [0,1] / [1,0,0,0,1] # \epsilon \partial_{x_{2}} x_{2}^{\mathrm{c}}
[0,0] / -1.0 / [0,1] / [1,0,0,2,0] # - \epsilon \partial_{x_{2}} \left(x_{1}^{\mathrm{c}}\right)^{2} x_{2}
[0,0] / 1.0 / [0,1] / [1,0,0,0,0] # \epsilon \partial_{x_{2}} x_{2}
[1,-1] / -2.0 / [0,1] / [1,0,0,1,1] # - 2 \eps